In [6]:
import numpy as np
import tensorflow as tf
from pydsmidi import Synthesizer, WriteMidi

# Define the parameters
sequence_length = 100  # Length of the input sequence
num_features = 128    # Number of possible musical notes
num_notes = 500       # Number of notes to generate

# Define the RNN model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(256, input_shape=(sequence_length, num_features), return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.LSTM(256, return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_features, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Generate music
def generate_music():
    # Initialize the seed sequence
    seed_sequence = np.random.rand(1, sequence_length, num_features)

    generated_music = []

    # Generate the desired number of notes
    for _ in range(num_notes):
        # Predict the next note probabilities
        predictions = model.predict(seed_sequence)
        
        # Sample the next note using the predicted probabilities
        next_note = np.random.choice(num_features, p=predictions[0][-1])

        # Append the next note to the generated music
        generated_music.append(next_note)

        # Update the seed sequence by shifting and adding the new note
        seed_sequence = np.concatenate((seed_sequence[:, 1:, :], np.expand_dims(predictions[:, -1, :], axis=1)), axis=1)

    return generated_music

# Generate music using the model
generated_music = generate_music()

# Create a synthesizer and write the generated music to a MIDI file
synth = Synthesizer()
for note in generated_music:
    synth.noteon(0, note, 127)
    synth.noteoff(0, note, 127)

WriteMidi("generated_music.mid", [synth.events], (num_notes * 480))



ModuleNotFoundError: No module named 'pydsmidi'

In [ ]:
import math

def gives_angle(perp_distance, eyes_distance):
    hypotenuse = math.sqrt(perp_distance**2 + eyes_distance**2)
    angle = math.atan(perp_distance/eyes_distance)
    return angle


In [ ]:
import cv2

# Constants for reference object size and distance
REFERENCE_OBJECT_WIDTH = 20  # Width of the reference object in centimeters
FOCAL_LENGTH = 550  # Focal length of the camera in pixels

def calculate_distance_to_object(object_width, focal_length, measured_width):
    # Calculate the distance to the object using the formula: distance = (reference_width * focal_length) / measured_width
    distance = (object_width * focal_length) / measured_width
    return distance

# Start the webcam
cap = cv2.VideoCapture(0)

# Load the face cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    # Read the current frame from the webcam
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = face_cascade.detectMultiScale(gray, 1.3, 10)

    # Iterate over detected faces
    for (x, y, w, h) in faces:
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        
        # Calculate the distance to the person's face
        distance = calculate_distance_to_object(REFERENCE_OBJECT_WIDTH, FOCAL_LENGTH, w)

        # Display the distance on the frame
        cv2.putText(frame, f"Distance: {distance:.2f} cm", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Distance Estimation', frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()
